0.5s) Error on fit parameters

A taylor expantion on our Friss equation gives

$$ \chi^2 = \sum_i (\frac{S_i-P_r^i(x,y)}{\sigma_i})^2\\
P_r^i(x,y) \approx P_r^i(x_0,y_0)+\frac{\partial P_r^i(x_0,y_0)}{\partial x}*(x-x_0)+\frac{\partial P_r^i(x_0,y_0)}{\partial y}*(y-y_0)\\
= P_0^i + P_{x,0}^i\Delta x + P_{y,0}^i\Delta y\\
\chi^2 = \sum_i (\frac{S_i-P_0^i + P_{x,0}^i\Delta x + P_{y,0}^i\Delta y}{\sigma_i})^2\\
S_i - P_0^i = R_0^i\\
\chi^2 = \sum_i \frac{1}{\sigma_i^2}((R_0^i)^2 + (P_{x,0}^i\Delta x)^2 + (P_{y,0}^i\Delta y)^2 + R_0^iP_{x,0}^i\Delta x + R_0^iP_{y,0}^i\Delta y + P_{x,0}^iP_{y,0}^i\Delta x\Delta y) $$

0.5t)

Using that the x and y derivatives go to zero, we get:

$$\sum_i \frac{1}{\sigma^2}R_0^iP_{x,0}^i = 0\\
\sum_i \frac{1}{\sigma^2}R_0^iP_{y,0}^i = 0\\
%\sum_i \frac{1}{\sigma^2}P_{x,0}^iP_{y,0}^i\Delta x\Delta y &\approx 0\\
\chi^2 = \sum_i \frac{1}{\sigma_i^2}((R_0^i)^2 + (P_{x,0}^i\Delta x)^2 + (P_{y,0}^i\Delta y)^2
                                      + P_{x,0}^iP_{y,0}^i\Delta x\Delta y)$$

0.5u)

Writing this as a matrix equation

In [3]:
%%latex

\begin{align*}
\chi^2 = \sum_i \frac{1}{\sigma_i^2}((R_0^i)^2 + (P_{x,0}^i\Delta x)^2 + (P_{y,0}^i\Delta y)^2
                                      + P_{x,0}^iP_{y,0}^i\Delta x\Delta y)\\
\chi^2 = \chi_0^2 + \sum_i \frac{1}{\sigma_i^2}((P_{x,0}^i\Delta x)^2 + (P_{y,0}^i\Delta y)^2
                                      + P_{x,0}^iP_{y,0}^i\Delta x\Delta y)\\
= A + 
\left(
\begin{array}{c}
\Delta x & \Delta y
\end{array}
\right)
* B *
\left(
\begin{array}{cc}
\Delta x \\
\Delta y
\end{array}
\right)\\

\left(
\begin{array}{c}
\Delta x & \Delta y
\end{array}
\right)

\left(
\begin{array}{cc}
a & b\\
c & d\\
\end{array}
\right) 

\left(
\begin{array}{cc}
\Delta x \\
\Delta y
\end{array}
\right)
= (\Delta x)^2 a + \Delta x\Delta y(b+c) + (\Delta y)^2 d\\
a = P_{x,0}^i\\
b + c = P_{x,0}^iP_{y,0}^i\\
b = c = \frac{1}{2}P_{x,0}^iP_{y,0}^i\\
d = P_{y,0}^i\\

A = \chi_0^2\\
B = 
\left(
\begin{array}{cc}
P_{x,0}^i & \frac{1}{2}P_{x,0}^iP_{y,0}^i\\
\frac{1}{2}P_{x,0}^iP_{y,0}^i & P_{y,0}^i\\
\end{array}
\right)

\end{align*}

<IPython.core.display.Latex object>

0.5v)

To find the maximal $\Delta x$ to raise $\chi^2$ by one, the $\Delta y$ and $\chi_0^2$ have to be zero. 
Also the $\Delta x$ will be maximal, if the complete $\chi^2$ value is caused by one router.

$$\chi^2 = \sum_i \frac{1}{\sigma_i^2}(P_{x,0}^i\Delta x)^2 = 1\\
\chi^2 = \frac{1}{\sigma_i^2}(P_{x,0}^i\Delta x)^2 = 1\\
(\frac{\sigma_i}{P_{x,0}^i})^2 = (\Delta x)^2\\
\Delta x = \pm \frac{\sigma_i}{P_{x,0}^i}\\
\text{The same procedure gives:}\\
\Delta y = \pm \frac{\sigma_i}{P_{y,0}^i}$$

0.5w) 
$$\frac{\partial P_r}{\partial x}= \frac{\partial}{\partial x}(P_t)+20 \times \log_{10}(\frac{c}{4\pi f})-20\log_{10}(r^2) )\\
= \frac{\partial}{\partial x} (-10 \times \frac{\ln (r^2)}{\ln (10)})\\
= \frac{-10}{\ln (10)}\times \frac{1}{r^2}\times \frac{\partial r^2 }{\partial x}\\
r = (x-x_i)^2+(y-y_i)^2+Z^2\\
\frac{\partial r^2}{\partial x} = 2(x-x_i) = 2 \Delta x\\
\Rightarrow \frac{\partial P_r}{\partial x}=\frac{-10}{\ln (10)} \times \frac{2\Delta x}{r^2}$$
and in the same way:
$$\Rightarrow \frac{\partial P_r}{\partial y}=\frac{-10}{\ln (10)} \times \frac{2\Delta y}{r^2}$$

In [4]:
import numpy as np
import math
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import numpy.random as rnd

# Constants
P_t = 0
c = 299792458
f = 2.4 * 10**9
Z = 2 # height of router - height of device

routers = [(0,0), (0,20), (20,20), (20, 0)]
S_x = 5
S_y = 5
sigma = 1

def P_r(r):
    # Friss function for actual signal at router
    P_r = P_t + 20 * np.log10(c / (4* np.pi* f* r))
    return P_r

def getR(S_x, S_y, x_i, y_i):
    # Get R from x-, y-, z values
    return math.sqrt((S_x - x_i)**2 + (S_y - y_i)**2 + Z**2)

def SiToR(Si):
    # Get radius from measured signal
    return c/(4*math.pi*f*10**(Si/20.0))
    
def calculate_S_is(S_x, S_y, routers, sigma):
    # Construct a measured signal from acual signal plus
    # gaussian noise
    Sis = []
    for (x_i, y_i) in routers:
        package_noise = np.random.normal(0, sigma, 1)[0]
        S_r = getR(S_x, S_y, x_i, y_i)
        S_i = P_r(S_r) + package_noise
        Sis.append(S_i)
    return Sis

def locationScore(xy):
    # Function that scores the goodness of the 
    # current xy position for optimize function
    x, y = xy
    score = 0
    for n, r in enumerate(routers):
        routerx = r[0]
        routery = r[1]
        tryRadial2 = (x - routerx)**2 + (y - routery)**2
        score += (radials[n]**2 - tryRadial2)**2
    return score

def getError(d, r):
    # Get error on one delta x or delta y (d) value
    # r is the radius
    return -20/math.log(10)*d/r

# Simulate wifi packet (plus noise) and 
# triangulate onto the position
Sis = calculate_S_is(S_x, S_y, routers, sigma)
radials = [SiToR(Si) for Si in Sis]
triangulate = minimize(locationScore, [10,10])
measuredx, measuredy = triangulate.x

# Calculate the x and y errors (for ellipses)
xerror, yerror = 0, 0
for router in routers:
    x, y = router
    deltax = x - measuredx
    deltay = y - measuredy
    r = getR(measuredx, measuredy, x, y)
    xerror += getError(deltax, r)
    yerror += getError(deltay, r)

# Plot ellipse
fig = plt.figure(0)
ax = fig.add_subplot(111, aspect='equal')
e = Ellipse(xy=(measuredx, measuredy), width=xerror, height=yerror)
e.set_alpha(0.3)
e.set_facecolor(rnd.rand(3))
ax.add_artist(e)

# Plot actual send point
ax.plot([S_x, measuredx], [S_y, measuredy], 'ro')

ax.set_xlim(0, 20)
ax.set_ylim(0, 20)
plt.show()